In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from collections import defaultdict

from surprise.model_selection import KFold

from surprise.model_selection.validation  import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans

import os
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

from surprise.model_selection import train_test_split

from sklearn.metrics import  roc_curve
from sklearn.metrics import roc_auc_score
from collections import defaultdict

from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.matrix_factorization import SVD

import seaborn as sns

import csv
import re


In [27]:
bike_data = pd.read_csv('../input/project4data/day.csv')

bike_data

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [28]:
video_data = pd.read_csv('../input/project4data/transcoding_mesurment.tsv', sep='\t')
video_data

,id,duration,codec,width,height,bitrate,framerate,i,p,b,...,p_size,b_size,size,o_codec,o_bitrate,o_framerate,o_width,o_height,umem,utime
0,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,176,144,22508,0.612
1,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,320,240,25164,0.980
2,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,480,360,29228,1.216
3,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,640,480,34316,1.692
4,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,1280,720,58528,3.456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68779,ZWEN-71BqPs,972.27100,h264,480,360,278822,29.000000,560,28580,0,...,26561730,0,33886358,flv,242000,24.00,640,480,88692,1.552
68780,zWQN-bqqg0o,129.88100,vp8,640,480,639331,30.162790,36,3855,0,...,9503846,0,10379630,mpeg4,539000,29.97,1920,1080,107524,18.557
68781,zX17-vi0sqQ,249.68000,vp8,320,240,359345,25.068274,129,6113,0,...,9456514,0,11215178,flv,539000,12.00,176,144,88708,0.752
68782,zyiT-TzxIpk,183.62334,h264,1280,720,2847539,29.000000,98,5405,0,...,60113035,0,65359329,mpeg4,539000,12.00,320,240,88724,5.444


In [29]:

bike_data_df = pd.DataFrame(data=bike_data)
video_data_df = pd.DataFrame(data=video_data)


In [ ]:
video_data_df.dtypes

In [ ]:
bike_data_df.dtypes

In [30]:
# one hot encoding - bike_data
from sklearn.preprocessing import OneHotEncoder 


#http://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example

#one hot encode weathersit for sure, season probably, weekday (0-6) maybe
# label encode the date into days of the month
encoded_bike_data = bike_data_df.copy()

encoded_bike_data['dteday'] = pd.to_datetime(bike_data_df['dteday']).dt.strftime('%d')
encoded_bike_data['dteday'] = pd.to_numeric(encoded_bike_data['dteday'])

# use pd.concat to join the new columns with your original dataframe
df_season = pd.concat([encoded_bike_data,pd.get_dummies(encoded_bike_data['season'], prefix='season')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_season.drop(['season'],axis=1, inplace=True)

df_weathersit = pd.concat([df_season, pd.get_dummies(df_season['weathersit'], prefix = 'weathersit')], axis=1)
df_weathersit.drop(['weathersit'], axis=1, inplace=True)


df_holiday = pd.concat([df_weathersit, pd.get_dummies(df_weathersit['holiday'], prefix = 'holiday')], axis=1)
df_holiday.drop(['holiday'], axis=1, inplace=True)

df_workday = pd.concat([df_holiday, pd.get_dummies(df_holiday['workingday'], prefix = 'workingday')], axis=1)
df_workday.drop(['workingday'], axis=1, inplace=True)
bike_data_proc_df = df_workday
print(bike_data_proc_df.columns)
bike_no_f_scaling = bike_data_proc_df.copy()
bike_data_proc_df


Index(['instant', 'dteday', 'yr', 'mnth', 'weekday', 'temp', 'atemp', 'hum',
       'windspeed', 'casual', 'registered', 'cnt', 'season_1', 'season_2',
       'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3',
       'holiday_0', 'holiday_1', 'workingday_0', 'workingday_1'],
      dtype='object')


,instant,dteday,yr,mnth,weekday,temp,atemp,hum,windspeed,casual,...,season_2,season_3,season_4,weathersit_1,weathersit_2,weathersit_3,holiday_0,holiday_1,workingday_0,workingday_1
0,1,1,0,1,6,0.344167,0.363625,0.805833,0.160446,331,...,0,0,0,0,1,0,1,0,1,0
1,2,2,0,1,0,0.363478,0.353739,0.696087,0.248539,131,...,0,0,0,0,1,0,1,0,1,0
2,3,3,0,1,1,0.196364,0.189405,0.437273,0.248309,120,...,0,0,0,1,0,0,1,0,0,1
3,4,4,0,1,2,0.200000,0.212122,0.590435,0.160296,108,...,0,0,0,1,0,0,1,0,0,1
4,5,5,0,1,3,0.226957,0.229270,0.436957,0.186900,82,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,27,1,12,4,0.254167,0.226642,0.652917,0.350133,247,...,0,0,0,0,1,0,1,0,0,1
727,728,28,1,12,5,0.253333,0.255046,0.590000,0.155471,644,...,0,0,0,0,1,0,1,0,0,1
728,729,29,1,12,6,0.253333,0.242400,0.752917,0.124383,159,...,0,0,0,0,1,0,1,0,1,0
729,730,30,1,12,0,0.255833,0.231700,0.483333,0.350754,364,...,0,0,0,1,0,0,1,0,1,0


In [ ]:
print(video_data_df.columns)
print(video_data)

In [31]:
# one hot encoding - video data
video_data_noid = video_data_df.copy()
video_data_noid.drop(['id'], axis=1, inplace=True)


df_codec = pd.concat([video_data_noid, pd.get_dummies(video_data_noid['codec'], prefix = 'codec')], axis=1)
df_codec.drop(['codec'], axis=1, inplace=True)


df_o_codec = pd.concat([df_codec, pd.get_dummies(df_codec['o_codec'], prefix = 'o_codec')], axis=1)
df_o_codec.drop(['o_codec'], axis=1, inplace=True)

video_data_proc_df = df_o_codec.copy()
video_no_f_scaling = video_data_proc_df.copy()
video_data_proc_df

,duration,width,height,bitrate,framerate,i,p,b,frames,i_size,...,umem,utime,codec_flv,codec_h264,codec_mpeg4,codec_vp8,o_codec_flv,o_codec_h264,o_codec_mpeg4,o_codec_vp8
0,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,22508,0.612,0,0,1,0,0,0,1,0
1,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,25164,0.980,0,0,1,0,0,0,1,0
2,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,29228,1.216,0,0,1,0,0,0,1,0
3,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,34316,1.692,0,0,1,0,0,0,1,0
4,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,58528,3.456,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68779,972.27100,480,360,278822,29.000000,560,28580,0,29140,7324628,...,88692,1.552,0,1,0,0,1,0,0,0
68780,129.88100,640,480,639331,30.162790,36,3855,0,3891,875784,...,107524,18.557,0,0,0,1,0,0,1,0
68781,249.68000,320,240,359345,25.068274,129,6113,0,6242,1758664,...,88708,0.752,0,0,0,1,1,0,0,0
68782,183.62334,1280,720,2847539,29.000000,98,5405,0,5503,5246294,...,88724,5.444,0,1,0,0,0,0,1,0


# Feature Scaling

In [32]:
#question 7 - bike data
from sklearn.preprocessing import StandardScaler

#standardize all feature columns

col_names = bike_data_proc_df.columns
features = bike_data_proc_df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

#assign scaled results to the dataset copy
bike_data_proc_df[col_names] = features
print(bike_data_proc_df)

print(np.mean(bike_data_proc_df['atemp']))


      instant    dteday        yr      mnth   weekday      temp     atemp  \
0   -1.729683 -1.674108 -1.001369 -1.600161  1.498809 -0.826662 -0.679946   
1   -1.724944 -1.560522 -1.001369 -1.600161 -1.496077 -0.721095 -0.740652   
2   -1.720205 -1.446936 -1.001369 -1.600161 -0.996930 -1.634657 -1.749767   
3   -1.715466 -1.333351 -1.001369 -1.600161 -0.497782 -1.614780 -1.610270   
4   -1.710728 -1.219765 -1.001369 -1.600161  0.001366 -1.467414 -1.504971   
..        ...       ...       ...       ...       ...       ...       ...   
726  1.710728  1.279122  0.998633  1.588660  0.500513 -1.318665 -1.521108   
727  1.715466  1.392707  0.998633  1.588660  0.999661 -1.323224 -1.346690   
728  1.720205  1.506293  0.998633  1.588660  1.498809 -1.323224 -1.424344   
729  1.724944  1.619879  0.998633  1.588660 -1.496077 -1.309558 -1.490049   
730  1.729683  1.733465  0.998633  1.588660 -0.996930 -1.528225 -1.540482   

          hum  windspeed    casual  ...  season_2  season_3  season_4  \
0 

In [33]:
#question 7 continued

from sklearn.preprocessing import StandardScaler


video_col_names = video_data_proc_df.columns
video_features = video_data_proc_df[video_col_names]
video_scaler = StandardScaler().fit(video_features.values)
video_features = video_scaler.transform(video_features.values)

#assign scaled results to the dataset copy
video_data_proc_df[video_col_names] = video_features
print(video_data_proc_df)

#check that mean of a column is approximately 0
print(np.mean(video_data_proc_df['p']))


       duration     width    height   bitrate  framerate         i         p  \
0     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
1     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
2     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
3     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
4     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
...         ...       ...       ...       ...        ...       ...       ...   
68779  2.387620 -0.312921 -0.218492 -0.378671   0.797072  5.416577  3.628857   
68780 -0.544926  0.032528  0.280232 -0.049625   0.958016 -0.765280 -0.440548   
68781 -0.127879 -0.658369 -0.717217 -0.305176   0.252873  0.331882 -0.068911   
68782 -0.357837  1.414323  1.277682  1.965863   0.797072 -0.033839 -0.185438   
68783  0.028544 -0.969273 -1.116197 -0.582738  -1.555936 -0.470344 -0.503255   

              b    frames    i_size  ..

In [34]:
#question 8 feature selection

from sklearn.feature_selection import f_regression, mutual_info_regression


#separate the matrices into features and targets
bike_target = bike_data_proc_df.cnt
video_target = video_data_proc_df.utime

bike_feature_mtx = bike_data_proc_df.drop(['registered','casual','cnt'], axis=1, inplace=False)

#account for dummy variable trap? that is leading to memory leak for video_data with f regression AND mutual info
#video_feature_mtx = video_data_proc_df.drop(['o_codec_flv', 'o_codec_h264', 'o_codec_mpeg4','o_codec_vp8','utime'], axis=1, inplace=False)
video_feature_mtx = video_data_proc_df.drop(['utime'], axis=1, inplace=False)


#f regression better for linear?
print("---- Bike Features ----")
bike_feature_r = f_regression(bike_feature_mtx, bike_target)
print("---- Bike F Regression ----")

print(bike_feature_r)

print("---- Bike Mutual Information ----")
#mutual information better for nonlinear?
bike_feature_mir = mutual_info_regression(bike_feature_mtx, bike_target, copy=True)
print(bike_feature_mir)


---- Bike Features ----
---- Bike F Regression ----
(array([4.76810740e+02, 4.27336147e-01, 3.44890586e+02, 6.20046245e+01,
       3.33109137e+00, 4.73471711e+02, 4.82454311e+02, 7.46194000e+00,
       4.24378416e+01, 3.38467598e+02, 1.59217364e+01, 9.94590876e+01,
       3.14999760e+00, 4.97987371e+01, 2.24590570e+01, 4.45403742e+01,
       3.42144104e+00, 3.42144104e+00, 2.73674228e+00, 2.73674228e+00]), array([1.02076318e-81, 5.13505103e-01, 2.48353990e-63, 1.24311178e-14,
       6.83908070e-02, 2.81062240e-81, 1.85450413e-82, 6.45414333e-03,
       1.35995868e-10, 2.22572389e-62, 7.26872835e-05, 4.79590176e-22,
       7.63443042e-02, 3.97113639e-12, 2.58147987e-06, 4.93394699e-11,
       6.47593579e-02, 6.47593579e-02, 9.84949616e-02, 9.84949616e-02]))
---- Bike Mutual Information ----
[0.90618392 0.         0.27512646 0.37543367 0.04580198 0.38946377
 0.46442778 0.04576237 0.05590348 0.19627569 0.0032439  0.0704746
 0.03497652 0.04306097 0.01874842 0.04359151 0.01152875 0.01152875

In [ ]:
print("---- Video Features ----")

#video_feature_r = f_regression(video_feature_mtx, video_target)
print("---- Video F Regression ----")

print(video_feature_r)

#mutual information better for nonlinear?
#print("---- Video Mutual Information ----")
#video_feature_mir = mutual_info_regression(video_feature_mtx, video_target, copy=True, n_neighbors=5)
#print(video_feature_mir)

#video mutual info still has some memory leak issue

In [ ]:
print(video_data_proc_df.columns)
print(video_feature_r)

In [35]:
#train ordinary least squares, lasso, ridge regression separately and compare

from sklearn import linear_model
from sklearn.model_selection  import cross_validate

#full processed sets
#bike_data_proc_df
#video_data_proc_df


#feature sets
#bike_feature_mtx
#video_feature_mtx

#target sets
#bike_target
#video_target


reg = linear_model.LinearRegression()
cv = cross_validate(reg, bike_feature_mtx, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
bike_train_MSE = -1* cv['test_score']
bike_test_MSE = -1* cv['train_score']

bike_train_RMSE = np.sqrt(bike_train_MSE)
bike_test_RMSE = np.sqrt(bike_test_MSE)

print("Linear Regression")
print("Bike Training RMSE: " +str(bike_train_RMSE))
print("Bike Testing RMSE: " +str(bike_test_RMSE))


print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))

print("---------------------------------")

reg = linear_model.LinearRegression(fit_intercept=False)
cv = cross_validate(reg, video_feature_mtx, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
# y is the target, x is the features
video_train_MSE = -1* cv['test_score']
video_test_MSE = -1* cv['train_score']

video_train_RMSE = np.sqrt(video_train_MSE)
video_test_RMSE = np.sqrt(video_test_MSE)



print("Training RMSE: " +str(video_train_RMSE))
print("Testing RMSE: " +str(video_test_RMSE))


print("Average Training RMSE: " +str(np.mean(video_train_RMSE)))
print("Average Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))

Linear Regression
Bike Training RMSE: [0.27523502 0.34314316 0.52274656 0.37870083 0.30681512 0.34204409
 0.49896039 0.56376921 0.61579663 0.76644172]
Bike Testing RMSE: [0.42793329 0.42131689 0.40611724 0.41822942 0.42475536 0.4239225
 0.40472221 0.39679186 0.39544988 0.38031118]
Average Bike Training RMSE: 0.46136527418917234
Average Bike Testing/Validation RMSE: 0.4099549844623903
---------------------------------
Training RMSE: [0.71068207 0.54537817 0.57032102 0.5862645  0.59814489 0.6467748
 0.53840222 0.55627409 0.52447296 0.86810586]
Testing RMSE: [0.57578102 0.59376065 0.59094268 0.58930588 0.58793159 0.58258828
 0.5950405  0.59278693 0.59670715 0.57878125]
Average Training RMSE: 0.614482059566546
Average Testing/Validation RMSE: 0.5883625935425374


In [36]:
#extract p values for bike - linear regression

import statsmodels.api as sm
mod = sm.OLS(bike_target,bike_feature_mtx)
fii = mod.fit()

p_values = fii.summary2().tables[1]['P>|t|']
print("Bike p-value data \n" +str(p_values))

mod_vid = sm.OLS(video_target,video_feature_mtx)
fii_vid = mod_vid.fit()

vid_p_values = fii_vid.summary2().tables[1]['P>|t|']
print("Video p-value data \n" +str(vid_p_values))

Bike p-value data 
instant         1.672256e-01
dteday          1.360059e-01
yr              1.951537e-01
mnth            1.649430e-01
weekday         9.424051e-06
temp            2.815635e-03
atemp           4.194957e-01
hum             8.871484e-05
windspeed       8.194246e-10
season_1        1.625252e-21
season_2        9.255108e-05
season_3        7.771212e-01
season_4        3.353098e-14
weathersit_1    8.284990e-14
weathersit_2    9.007087e-04
weathersit_3    4.544722e-18
holiday_0       3.568426e-03
holiday_1       3.568426e-03
workingday_0    8.230212e-02
workingday_1    8.230212e-02
Name: P>|t|, dtype: float64
Video p-value data 
duration         4.923825e-03
width            1.740821e-30
height           9.375400e-27
bitrate          1.870629e-99
framerate        8.298052e-14
i                1.198038e-01
p                1.198038e-01
b                1.198038e-01
frames           1.198038e-01
i_size           1.221039e-01
p_size           1.812032e-01
b_size           1.1980

In [37]:
# lasso/l1 regression


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

#bike lasso regression
for alphas in alpha_values:
    
    lass = linear_model.Lasso(alpha=alphas)
    cv = cross_validate(lass, bike_feature_mtx, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Lasso Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video lasso regression
for alphas in alpha_values:
    lass = linear_model.Lasso(alpha=alphas, fit_intercept=False)
    cv = cross_validate(lass, video_feature_mtx, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Lasso Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11187376731726317, tolerance: 0.05331093559749604
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6883697346504505, tolerance: 0.0653775140172978
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09544176693637496, tolerance: 0.05740875566762194
  positive)


Lasso Regression, Alpha = 0.0001
Average Bike Training RMSE: 0.46290641561055557
Average Bike Testing/Validation RMSE: 0.4104649485720011
---------------------------------
Lasso Regression, Alpha = 0.001
Average Bike Training RMSE: 0.4622293053525649
Average Bike Testing/Validation RMSE: 0.41053018396641716
---------------------------------
Lasso Regression, Alpha = 0.01
Average Bike Training RMSE: 0.4562246154502545
Average Bike Testing/Validation RMSE: 0.41277439398717536
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.058682800617390285, tolerance: 0.05740875566762194
  positive)


Lasso Regression, Alpha = 0.1
Average Bike Training RMSE: 0.5295080476607967
Average Bike Testing/Validation RMSE: 0.48345670627648146
---------------------------------
Lasso Regression, Alpha = 1
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------
Lasso Regression, Alpha = 10
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------
Lasso Regression, Alpha = 100
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------
Lasso Regression, Alpha = 1000
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.084753725317569, tolerance: 6.1798795812682545
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.262155339059973, tolerance: 6.184145996353777
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.232537513646093, tolerance: 6.117027207774886
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.229332219

Lasso Regression, Alpha = 0.0001
Average Video Training RMSE: 0.5965738035085592
Average Video Testing/Validation RMSE: 0.5882936388021214
---------------------------------
Lasso Regression, Alpha = 0.001
Average Video Training RMSE: 0.5960967021825969
Average Video Testing/Validation RMSE: 0.5886350031747362
---------------------------------
Lasso Regression, Alpha = 0.01
Average Video Training RMSE: 0.5936353399637894
Average Video Testing/Validation RMSE: 0.5906030380425223
---------------------------------
Lasso Regression, Alpha = 0.1
Average Video Training RMSE: 0.6489622510375124
Average Video Testing/Validation RMSE: 0.6474615493665328
---------------------------------
Lasso Regression, Alpha = 1
Average Video Training RMSE: 0.9961400453096696
Average Video Testing/Validation RMSE: 0.9999586275305898
---------------------------------
Lasso Regression, Alpha = 10
Average Video Training RMSE: 0.9961400453096696
Average Video Testing/Validation RMSE: 0.9999586275305898
-----------

In [38]:
#ridge/l2 regression


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

#bike ridge regression
for alphas in alpha_values:
    
    ridg = linear_model.Ridge(alpha=alphas)
    cv = cross_validate(ridg, bike_feature_mtx, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Ridge Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video ridge regression
for alphas in alpha_values:
    ridg = linear_model.Ridge(alpha=alphas, fit_intercept=True)
    cv = cross_validate(ridg, video_feature_mtx, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Ridge Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

Ridge Regression, Alpha = 0.0001
Average Bike Training RMSE: 0.4615732593051864
Average Bike Testing/Validation RMSE: 0.40995481876532774
---------------------------------
Ridge Regression, Alpha = 0.001
Average Bike Training RMSE: 0.4621284763941973
Average Bike Testing/Validation RMSE: 0.41003347310239774
---------------------------------
Ridge Regression, Alpha = 0.01
Average Bike Training RMSE: 0.46284797078027484
Average Bike Testing/Validation RMSE: 0.4103490244762412
---------------------------------
Ridge Regression, Alpha = 0.1
Average Bike Training RMSE: 0.4629792268297678
Average Bike Testing/Validation RMSE: 0.4104693369599798
---------------------------------
Ridge Regression, Alpha = 1
Average Bike Training RMSE: 0.46261848665318617
Average Bike Testing/Validation RMSE: 0.4104980674184879
---------------------------------
Ridge Regression, Alpha = 10
Average Bike Training RMSE: 0.4594811779519955
Average Bike Testing/Validation RMSE: 0.4108191892325195
-------------------

# **Linear Regression without feature scaling**

In [39]:
#no scaling features/targets

bike_no_f_target = bike_no_f_scaling.cnt
video_no_f_target = video_no_f_scaling.utime

bike_feature_no_f_mtx = bike_no_f_scaling.drop(['registered','casual','cnt'], axis=1, inplace=False)

#account for dummy variable trap? that is leading to memory leak for video_data with f regression AND mutual info
video_feature_no_f_mtx = video_no_f_scaling.drop(['utime'], axis=1, inplace=False)

In [40]:
#train ordinary least squares, lasso, ridge regression separately and compare

from sklearn import linear_model
from sklearn.model_selection  import cross_validate

#full processed sets
#bike_no_f_scaling
#video_no_f_scaling


#feature sets
#bike_feature_no_f_mtx
#video_feature_no_f_mtx

#target sets
#bike_no_f_target
#video_no_f_target


reg = linear_model.LinearRegression()
cv = cross_validate(reg, bike_feature_no_f_mtx, bike_no_f_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
bike_train_MSE = -1* cv['test_score']
bike_test_MSE = -1* cv['train_score']

bike_train_RMSE = np.sqrt(bike_train_MSE)
bike_test_RMSE = np.sqrt(bike_test_MSE)

print("NO FEATURE SCALING")
print("Linear Regression")
print("Bike Training RMSE: " +str(bike_train_RMSE))
print("Bike Testing RMSE: " +str(bike_test_RMSE))


print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))

print("---------------------------------")

reg = linear_model.LinearRegression(fit_intercept=False)
cv = cross_validate(reg, video_feature_no_f_mtx, video_no_f_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
# y is the target, x is the features
video_train_MSE = -1* cv['test_score']
video_test_MSE = -1* cv['train_score']

video_train_RMSE = np.sqrt(video_train_MSE)
video_test_RMSE = np.sqrt(video_test_MSE)



print("Training RMSE: " +str(video_train_RMSE))
print("Testing RMSE: " +str(video_test_RMSE))


print("Average Training RMSE: " +str(np.mean(video_train_RMSE)))
print("Average Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))

NO FEATURE SCALING
Linear Regression
Bike Training RMSE: [ 532.82360767  664.28602663 1011.97772284  733.12162534  593.95907692
  662.15835052  965.93041857 1091.39290362 1194.22981225 1483.74377081]
Bike Testing RMSE: [828.43005368 815.6214587  786.19666886 809.64446687 822.2779376
 820.66561889 783.49603391 768.14378815 765.50411813 736.23907695]
Average Bike Training RMSE: 893.3623315191404
Average Bike Testing/Validation RMSE: 793.6219221739186
---------------------------------
Training RMSE: [12.05965581  8.78125706  9.19161965  9.44837759  9.65677112 10.40852618
  8.72314384  8.96349004  8.60155002 11.20796899]
Testing RMSE: [9.23824918 9.56368532 9.51799868 9.49169204 9.46863321 9.38261331
 9.58201139 9.54718882 9.6038961  9.32146776]
Average Training RMSE: 9.704236029567287
Average Testing/Validation RMSE: 9.471743581415199


In [41]:
# lasso/l1 regression NO FEATURE SCALING


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10]

#bike lasso regression
for alphas in alpha_values:
    
    lass = linear_model.Lasso(alpha=alphas)
    cv = cross_validate(lass, bike_feature_no_f_mtx, bike_no_f_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Lasso Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video lasso regression
for alphas in alpha_values:
    lass = linear_model.Lasso(alpha=alphas, fit_intercept=False)
    cv = cross_validate(lass, video_feature_no_f_mtx, video_no_f_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Lasso Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

Lasso Regression, Alpha = 0.0001
Average Bike Training RMSE: 896.1492474128066
Average Bike Testing/Validation RMSE: 794.6025096755498
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 225743586.52754876, tolerance: 199790.9642304414
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 219466602.2581918, tolerance: 255245.10859148938
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 203635064.0824722, tolerance: 271504.2992972645
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 215969358.

Lasso Regression, Alpha = 0.001
Average Bike Training RMSE: 896.1475259789829
Average Bike Testing/Validation RMSE: 794.602378590956
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 223160535.58523062, tolerance: 252850.1612019757
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 221988673.44637537, tolerance: 259618.5997721885
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 203474279.3406549, tolerance: 245012.33036170216
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 194467841

Lasso Regression, Alpha = 0.01
Average Bike Training RMSE: 896.1300470106928
Average Bike Testing/Validation RMSE: 794.6011012004999
---------------------------------


...

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3781188.4218069175, tolerance: 2211.728448591201
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3895649.814002646, tolerance: 2211.964603064701
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3869458.215883846, tolerance: 2221.9670914086005
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3819839.39

Lasso Regression, Alpha = 1
Average Video Training RMSE: 10.991848340569438
Average Video Testing/Validation RMSE: 10.720902763022131
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4019435.73588389, tolerance: 2211.728448591201
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4087794.664387365, tolerance: 2211.964603064701
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3530109.839321128, tolerance: 2221.9670914086005
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3953722.9073

Lasso Regression, Alpha = 10
Average Video Training RMSE: 11.481348480584554
Average Video Testing/Validation RMSE: 11.300996120583715
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3953255.425043078, tolerance: 2208.8288230415
  positive)


In [42]:
#ridge/l2 regression NO FEATURE SCALING


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10]

#bike ridge regression
for alphas in alpha_values:
    
    ridg = linear_model.Ridge(alpha=alphas)
    cv = cross_validate(ridg, bike_feature_no_f_mtx, bike_no_f_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Ridge Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video ridge regression
for alphas in alpha_values:
    ridg = linear_model.Ridge(alpha=alphas, fit_intercept=False)
    cv = cross_validate(ridg, video_feature_no_f_mtx, video_no_f_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Ridge Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

Ridge Regression, Alpha = 0.0001
Average Bike Training RMSE: 893.616164458566
Average Bike Testing/Validation RMSE: 793.629619095929
---------------------------------
Ridge Regression, Alpha = 0.001
Average Bike Training RMSE: 894.7618590019941
Average Bike Testing/Validation RMSE: 793.8524961926136
---------------------------------
Ridge Regression, Alpha = 0.01
Average Bike Training RMSE: 895.7244308408448
Average Bike Testing/Validation RMSE: 794.4123699779782
---------------------------------
Ridge Regression, Alpha = 0.1
Average Bike Training RMSE: 895.3315058833584
Average Bike Testing/Validation RMSE: 794.6961408867923
---------------------------------
Ridge Regression, Alpha = 1
Average Bike Training RMSE: 893.9781747394675
Average Bike Testing/Validation RMSE: 797.3086063373864
---------------------------------
Ridge Regression, Alpha = 10
Average Bike Training RMSE: 923.1669440736553
Average Bike Testing/Validation RMSE: 840.6680395972256
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04075e-25): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.14308e-25): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.34314e-25): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.05209e-25): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04127e-24): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAl

Ridge Regression, Alpha = 0.0001
Average Video Training RMSE: 20.574706975912044
Average Video Testing/Validation RMSE: 9.471743581415215
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04671e-24): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.05514e-24): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.05216e-24): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04294e-23): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.13848e-23): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAl

Ridge Regression, Alpha = 0.001
Average Video Training RMSE: 10.089329519424819
Average Video Testing/Validation RMSE: 9.471743581417353
---------------------------------


...

Ridge Regression, Alpha = 0.1
Average Video Training RMSE: 9.70218863635826
Average Video Testing/Validation RMSE: 9.471743602939934
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04294e-21): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.13848e-21): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04077e-21): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.14232e-21): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.54698e-21): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAl

Ridge Regression, Alpha = 1
Average Video Training RMSE: 9.703422514896962
Average Video Testing/Validation RMSE: 9.471745724090775
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04289e-20): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.13848e-20): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.04075e-20): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.14287e-20): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.54729e-20): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAl

Ridge Regression, Alpha = 10
Average Video Training RMSE: 9.71642680742256
Average Video Testing/Validation RMSE: 9.471948412352178
---------------------------------


In [43]:
# polynomial regression
#example: https://towardsdatascience.com/polynomial-regression-bbe8b9d97491 

from sklearn.preprocessing import PolynomialFeatures


reg = linear_model.LinearRegression()

poly = PolynomialFeatures(2)
bike_feature_transf = poly.fit_transform(bike_feature_mtx)


cv = cross_validate(reg, bike_feature_transf, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
bike_train_MSE = -1* cv['test_score']
bike_test_MSE = -1* cv['train_score']

bike_train_RMSE = np.sqrt(bike_train_MSE)
bike_test_RMSE = np.sqrt(bike_test_MSE)

print("Polynomial Regression")
print("Bike Training RMSE: " +str(bike_train_RMSE))
print("Bike Testing RMSE: " +str(bike_test_RMSE))


print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))

print("---------------------------------")

poly = PolynomialFeatures(2)
video_feature_transf = poly.fit_transform(video_feature_mtx)


reg = linear_model.LinearRegression(fit_intercept=False)
cv = cross_validate(reg, video_feature_transf, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
# y is the target, x is the features
video_train_MSE = -1* cv['test_score']
video_test_MSE = -1* cv['train_score']

video_train_RMSE = np.sqrt(video_train_MSE)
video_test_RMSE = np.sqrt(video_test_MSE)



print("Training RMSE: " +str(video_train_RMSE))
print("Testing RMSE: " +str(video_test_RMSE))


print("Average Training RMSE: " +str(np.mean(video_train_RMSE)))
print("Average Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))

Polynomial Regression
Bike Training RMSE: [1.22258793 0.82867659 0.44087674 0.6087659  0.90234915 0.53574766
 0.62481162 0.49422033 0.7125321  0.80283383]
Bike Testing RMSE: [0.26662085 0.25042069 0.2667929  0.2571286  0.25580243 0.26177871
 0.23850609 0.25312314 0.25192993 0.23965192]
Average Bike Training RMSE: 0.7173401850052193
Average Bike Testing/Validation RMSE: 0.25417552520247594
---------------------------------
Training RMSE: [4.50603015e-01 3.88181978e-01 3.28949779e-01 3.24556368e-01
 5.62501107e-01 3.67817554e-01 3.50587680e-01 3.38502112e-01
 7.22385278e+10 5.38679481e+12]
Testing RMSE: [0.31146662 0.32575642 0.32357634 0.32506191 0.31620015 0.31857372
 0.32441121 0.32416139 0.31696089 0.30775914]
Average Training RMSE: 545903333351.1984
Average Testing/Validation RMSE: 0.31939277984259495


In [44]:
# polynomial regression with regularization

# lasso/l1 regression


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
poly = PolynomialFeatures(2)
bike_feature_transf = poly.fit_transform(bike_feature_mtx)
video_feature_transf = poly.fit_transform(video_feature_mtx)

#bike lasso regression
for alphas in alpha_values:
    lass = linear_model.Lasso(alpha=alphas)
    cv = cross_validate(lass, bike_feature_transf, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Polynomial Lasso Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video lasso regression
for alphas in alpha_values:
    lass = linear_model.Lasso(alpha=alphas, fit_intercept=False)
    cv = cross_validate(lass, video_feature_transf, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Polynomial Lasso Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12.205061481414663, tolerance: 0.05331093559749604
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.06359888678999, tolerance: 0.06810796273049605
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.102130804009214, tolerance: 0.07244646057959403
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.874

Polynomial Lasso Regression, Alpha = 0.0001
Average Bike Training RMSE: 0.6538849003481674
Average Bike Testing/Validation RMSE: 0.26764268383773426
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4651692616325249, tolerance: 0.06810796273049605
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3736753036033491, tolerance: 0.07244646057959403
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4756386039667788, tolerance: 0.07121103476629716
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4

Polynomial Lasso Regression, Alpha = 0.001
Average Bike Training RMSE: 0.5557127037312617
Average Bike Testing/Validation RMSE: 0.2758804281974102
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06290806033895535, tolerance: 0.061403872386564924
  positive)


...

Polynomial Lasso Regression, Alpha = 0.0001
Average Video Training RMSE: 0.5492232366166651
Average Video Testing/Validation RMSE: 0.32282873102993687
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 123.68884236036138, tolerance: 6.13428952795074
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.39027882828668, tolerance: 6.1798795812682545
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.624281976690327, tolerance: 6.184145996353777
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 177.54887

Polynomial Lasso Regression, Alpha = 0.001
Average Video Training RMSE: 0.4165297602119706
Average Video Testing/Validation RMSE: 0.32785185224313046
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.350900185166211, tolerance: 6.3019381035821604
  positive)


Polynomial Lasso Regression, Alpha = 0.01
Average Video Training RMSE: 0.3747687077552639
Average Video Testing/Validation RMSE: 0.3491887215120602
---------------------------------
Polynomial Lasso Regression, Alpha = 0.1
Average Video Training RMSE: 0.5383986397307855
Average Video Testing/Validation RMSE: 0.5325404586577137
---------------------------------
Polynomial Lasso Regression, Alpha = 1
Average Video Training RMSE: 0.8198509246667222
Average Video Testing/Validation RMSE: 0.8227078704968805
---------------------------------
Polynomial Lasso Regression, Alpha = 10
Average Video Training RMSE: 0.9961400453096696
Average Video Testing/Validation RMSE: 0.9999586275305898
---------------------------------
Polynomial Lasso Regression, Alpha = 100
Average Video Training RMSE: 0.9961400453096696
Average Video Testing/Validation RMSE: 0.9999586275305898
---------------------------------
Polynomial Lasso Regression, Alpha = 1000
Average Video Training RMSE: 0.9961400453096696
Average

In [45]:
#ridge/l2 regression POLYNOMIAL


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

poly = PolynomialFeatures(2)
bike_feature_transf = poly.fit_transform(bike_feature_mtx)
video_feature_transf = poly.fit_transform(video_feature_mtx)

#bike ridge regression
for alphas in alpha_values:
    
    ridg = linear_model.Ridge(alpha=alphas)
    cv = cross_validate(ridg, bike_feature_transf, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Polynomial Ridge Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video ridge regression
for alphas in alpha_values:
    ridg = linear_model.Ridge(alpha=alphas, fit_intercept=False)
    cv = cross_validate(ridg, video_feature_transf, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Polynomial Ridge Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

Polynomial Ridge Regression, Alpha = 0.0001
Average Bike Training RMSE: 0.8037081615588468
Average Bike Testing/Validation RMSE: 0.25670951027399325
---------------------------------
Polynomial Ridge Regression, Alpha = 0.001
Average Bike Training RMSE: 0.7390692439672109
Average Bike Testing/Validation RMSE: 0.26055073432607706
---------------------------------
Polynomial Ridge Regression, Alpha = 0.01
Average Bike Training RMSE: 0.704365654121111
Average Bike Testing/Validation RMSE: 0.2648175654373136
---------------------------------
Polynomial Ridge Regression, Alpha = 0.1
Average Bike Training RMSE: 0.6872013976689418
Average Bike Testing/Validation RMSE: 0.26632882576847017
---------------------------------
Polynomial Ridge Regression, Alpha = 1
Average Bike Training RMSE: 0.6413122986116178
Average Bike Testing/Validation RMSE: 0.26972968371333694
---------------------------------
Polynomial Ridge Regression, Alpha = 10
Average Bike Training RMSE: 0.5565173985030689
Average Bik

In [46]:
# grid search with perceptron neural net

#
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from scipy import stats



multilayer = MLPRegressor()
parameter_space = {
    'hidden_layer_sizes': [(50,50), (50,100,50), (20,20,20,50), (20,20,20,20,50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.05, 0.01, 0.1],
    'learning_rate': ['adaptive'],
}
clf_bike = GridSearchCV(multilayer, parameter_space, n_jobs=-1, cv=10, return_train_score=True, scoring='neg_mean_squared_error')
clf_bike.fit(bike_feature_mtx, bike_target)

print('Bike Data: Best MLP parameters found:\n', clf_bike.best_params_)

Bike Data: Best MLP parameters found:
 {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [51]:
print(clf_bike.best_score_)
bike_train_MSE = -1* clf_bike['test_score']
bike_test_MSE = -1* clf_bike['train_score']

bike_train_RMSE = np.sqrt(bike_train_MSE)
bike_test_RMSE = np.sqrt(bike_test_MSE)

print("Average Bike Training Data MLP with optimal hyper-parameters, RMSE: " +str(np.mean(bike_train_RMSE)))
print("Average Bike Test Data MLP with optimal hyper-parameters, RMSE: " +str(np.mean(bike_test_RMSE)))


-0.1809007811924734


TypeError: 'GridSearchCV' object is not subscriptable

In [ ]:
print("Best Bike Test RMSE Multilayer Perceptron: "+str(np.sqrt(-1* clf_bike.best_score_)))

In [50]:
# grid search with perceptron neural net
#Video dataset


multilayer = MLPRegressor()
parameter_space = {
    'hidden_layer_sizes': [(50,50), (50,100,50), (20,20,20,50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.05, 0.01, 0.1],
    'learning_rate': ['adaptive'],
}
clf_video = GridSearchCV(multilayer, parameter_space, n_jobs=-1, cv=10, return_train_score=True,scoring='neg_mean_squared_error')
clf_video.fit(video_feature_mtx, video_target)

print('Video Data: Best MLP parameters found:\n', clf_video.best_params_)

KeyboardInterrupt: 

In [ ]:
print(clf_video.best_score_)
video_train_MSE = -1* clf_video['test_score']
video_test_MSE = -1* clf_video['train_score']

video_train_RMSE = np.sqrt(video_train_MSE)
video_test_RMSE = np.sqrt(video_test_MSE)

print("Video Training Data MLP with optimal hyper-parameters, RMSE: " +str(np.mean(video_train_RMSE)))
print("Video Test Data MLP with optimal hyper-parameters, RMSE: " +str(np.mean(video_test_RMSE)))


In [ ]:
#comment